# Cleaning FiveThirtyEight Biden Approval Rating Poll Data
###### Parsing start/end dates using a combo of Python and OfficeLibre Calc (spreadsheet manipulation)

In [1]:
import calendar
import datetime

import pandas as pd

In [2]:
months = {month.upper(): index for index, month in enumerate(calendar.month_abbr) if month}


In [3]:

def find_start_end_date(datestring):
    if not datestring:
        return '', ''
    
    else:
        count_letters = 0
        if type(datestring)==str:
            for digit in datestring:
                if digit.isalpha():
                    count_letters += 1
            if count_letters==3:
                month, dayrange, year = datestring.split()
                month_num = months[month.strip(".")]
                start_str, end_str = dayrange.split("-")
                start_num, end_num, year_num = int(start_str), int(end_str), int(year)

                start = datetime.date(year=year_num, month=month_num, day=start_num)
                end = datetime.date(year=year_num, month=month_num, day=end_num)
                return start, end

            elif count_letters==6:
                if datestring[-5] == "-":
                    datestring = datestring[:-5]
                year_num = int(datestring[-4:])
                start_datestring, end_datestring = datestring.split("-")
                start_month, start_day = start_datestring.split()
                start_month_num = months[start_month.strip(".")]
                start_day_num = int(start_day)
                end_month, end_day = end_datestring[:-4].split()
                end_month_num = months[end_month.strip(".")]
                end_day_num = int(end_day)

                start = datetime.date(year=year_num, month=start_month_num, day=start_day_num)
                end = datetime.date(year=year_num, month=end_month_num, day=end_day_num)
                return start, end
            else:
                return "this is weird: " + datestring , "this is weird: " + datestring
    return '', ''

In [4]:
biden_approval_df = pd.read_csv('data/BidenApprovalData.csv')
biden_approval_df['datestring'] = biden_approval_df['DATES'] + " " + biden_approval_df['YEAR']
biden_approval_df['bothdates'] = biden_approval_df['datestring'].apply(find_start_end_date)
biden_approval_df.dropna(how='all', inplace=True)
biden_approval_df['start'] = biden_approval_df['bothdates'].apply(lambda x: list(x)[0])
biden_approval_df['end'] = biden_approval_df['bothdates'].apply(lambda x: list(x)[-1])

In [5]:
biden_approval_df.to_csv("data/interim_cleaned_data.csv", index=False)

###### Explode date ranges to individual rows (following interim step to fill in single-day poll start/end dates by hand)

In [6]:
biden_approval_df = pd.read_csv("data/interim_handcleaned_data.csv")

In [7]:
biden_approval_df['clean_date'] = [pd.date_range(s, e, freq='d') for s, e in
              zip(pd.to_datetime(biden_approval_df['start']),
                  pd.to_datetime(biden_approval_df['end']))]

biden_approval_df = biden_approval_df.explode('clean_date').drop(['start', 'end'], axis=1)

###### Fixing approval and disapproval values

In [8]:
biden_approval_df["APPROVE"] = biden_approval_df["APPROVE"].apply(lambda x: float(x.strip("%")))
biden_approval_df["DISAPPROVE"] = biden_approval_df["DISAPPROVE"].apply(lambda x: float(x.strip("%")))
biden_approval_df["ADJUSTED_APPROVE"] = biden_approval_df["ADJUSTED_APPROVE"].apply(lambda x: float(x.strip("%")))
biden_approval_df["ADJUSTED_DISAPPROVE"] = biden_approval_df["ADJUSTED_DISAPPROVE"].apply(lambda x: float(x.strip("%")))



In [9]:
biden_daily_approvals = biden_approval_df.groupby("clean_date").mean()
biden_daily_approvals['day'] = biden_daily_approvals.index.day
biden_daily_approvals['month'] = biden_daily_approvals.index.month
biden_daily_approvals['year'] = biden_daily_approvals.index.year

In [10]:
biden_daily_approvals

,SAMPLE,WEIGHT,APPROVE,DISAPPROVE,ADJUSTED_APPROVE,ADJUSTED_DISAPPROVE,day,month,year
clean_date,,,,,,,,,
2021-01-20,4126.000000,0.824000,52.400000,30.400000,51.800000,32.000000,20,1,2021
2021-01-21,2598.600000,0.990000,54.200000,33.500000,53.700000,33.800000,21,1,2021
2021-01-22,2671.600000,0.907000,55.800000,34.400000,55.100000,34.800000,22,1,2021
2021-01-23,2863.888889,0.904444,55.222222,34.333333,54.888889,34.666667,23,1,2021
2021-01-24,2738.800000,1.053000,53.900000,34.500000,53.900000,34.800000,24,1,2021
...,...,...,...,...,...,...,...,...,...
2022-04-01,1810.500000,0.635000,40.500000,54.000000,40.500000,56.000000,1,4,2022
2022-04-02,1707.000000,0.790000,40.666667,52.666667,41.000000,54.333333,2,4,2022
2022-04-03,1655.250000,0.950000,41.000000,53.500000,41.750000,53.250000,3,4,2022


In [11]:
biden_daily_approvals.columns

Index(['SAMPLE', 'WEIGHT', 'APPROVE', 'DISAPPROVE', 'ADJUSTED_APPROVE',
       'ADJUSTED_DISAPPROVE', 'day', 'month', 'year'],
      dtype='object')

In [12]:
biden_daily_approvals.to_csv('data/biden_daily_approvals.csv')